Sudoku solver - I will test it on a csv file of 80k+ examples I found on Kaggle. I define classes for the overall grid, for each cell and for each row, column and 3x3 square. The row, column and square classes inherit from the group class.

Taking a 9x9 grid as an example, the rows are numbered from top to bottom (0-8), the columns from left to right (0-8) and the squares from top left to bottom right as below

0 - 1 - 2

3 - 4 - 5

6 - 7 - 8

The cells are numbered in the same fashion as the squares - top left to bottom right(0-80).

In [ ]:
import numpy as np
import pandas as pd
import time
from tqdm.notebook import tqdm
from google.colab import output
from copy import deepcopy

# grid class
class grid:
  def __init__(self,dim,render_at_each_step = False):
    self.dim = dim
    self.sqrt_dim = int(np.sqrt(self.dim))
    self.columns = []
    self.rows = []
    self.squares = []
    self.cells = []
    self.solvable = True
    self.render_at_each_step = render_at_each_step
    
    # set up the grid object
    # grid knows about the rows, columns, squares and cells
        
    
    # there are #dim rows, columns and squares
    for i in range(self.dim):
      self.columns.append(column(self,i))
      self.rows.append(row(self,i))
      self.squares.append(square(self,i))
    
    # there are dim**2 cells
    for i in range(self.dim):
      for j in range(self.dim):
        self.cells.append(cell(self,i*self.dim+j))
    
    # rows, columns and squares know which cells belong to them
    for i,c in enumerate(self.columns):
      self.columns[i] = c.find_cells()
  
    for i,r in enumerate(self.rows):
      self.rows[i] = r.find_cells()

    for i,s in enumerate(self.squares):
      self.squares[i] = s.find_cells()

  # visualize the current grid
  def render(self):
    for i, r in enumerate(self.rows):
      if i in [3, 6]:
        print('------+------+------')
      for j, c in enumerate(self.columns):
        if j in [3, 6]:
          print('|',end='')
        val = self.cells[i*self.dim+j].value
        if np.isnan(val):
          print(' ',end=' ')
        else:
          print(self.cells[i*self.dim+j].value,end=' ')
      print()


  # output the grid to a single string, for checking results are correct
  def to_string(self):
    vals = np.array([c.value for c in self.cells])
    vals[np.isnan(vals)] = 0
    s = ''
    for v in vals:
      s=s+str(int(v))    
    return s

  def solved(self):
    # returns true when all cells are allocated
    return  not np.any(np.isnan(np.array([c.value for c in self.cells])))

# cell class
class cell:
  def __init__(self,grid,number,val=np.NaN):
    
    self.number = number #cells are numbers from 0 to dim**2 - 1
    
    self.grid = grid

    self.value = val # can be 0 to dim-1 or np.nan (meaning a value has yet to be assigned)

    # each cell should have a reference to its square, row and column
    self.row_no = int(np.floor(self.number / self.grid.dim)) # row number = floor(num/dim)
    self.row = self.grid.rows[self.row_no]

    self.column_no = int(np.mod(self.number , self.grid.dim)) # column numbe = number mod dim
    self.column = self.grid.columns[self.column_no]

    # this is a bit more complicated ... 
    self.square_no = int(np.floor(self.row_no / self.grid.sqrt_dim) * self.grid.sqrt_dim +\
      np.floor(self.column_no / self.grid.sqrt_dim))
    self.square = self.grid.squares[self.square_no]

    # an array of dim bools, initially all true
    # we set these to false if the corresponding value already appears elsewhere in the relevant row, column or square
    self.can_be = np.full([self.grid.dim],True) #array of bools - intialize to all true 

  def set_value(self,value):
    if np.isnan(self.value):
      self.value = value
      self.can_be = np.full([self.grid.dim],False)
      self.can_be[value-1] = True
      
      # refresh the row, column and square
      self.row = self.row.refresh()
      self.column = self.column.refresh()
      self.square = self.square.refresh()

      if self.grid.render_at_each_step:
        output.clear()
        self.grid.render()
        time.sleep(0.5)
      return self


  def sanity(self):
    # if a cell can't take any of the allowed values, something is wrong
    # the grid may be inconsistent
    if np.sum(self.can_be==False) == self.grid.dim:
      self.grid.solvable = False

  def refresh(self):
    
    if not self.grid.solved():

      # only call this if the cell is not already assigned
      # this cell cant take the value of any other cell in its column
      if np.isnan(self.value):
        for v in self.column.contains:
          if not np.isnan(v):
            self.can_be[v-1] = False

      # ... or its row
        for v in self.row.contains:
          if not np.isnan(v):
            self.can_be[v-1] = False

      # ... or its square
        for v in self.square.contains:
          if not np.isnan(v):
            self.can_be[v-1] = False
      
    return self

# row, column and square inherit from the group class
class group:
  def __init__(self,grid,number):
    self.number = number
    self.grid = grid
    self.cells = []
    self.contains = [] # the list of numbers already assigned to this group
  
  # this function has the same implementation for rows, columns or squares
  def check(self):
    if not self.grid.solved():
    
      #perform 2 checks
      
      # if there is only one number missing from this group we can fill it in
      #print(self.__class__)
      #print(self.number)
      #print(self.contains)
      self.contains = [c.value for c in self.cells]
      if len(self.contains) ==  self.grid.dim-1:
        missing_value = np.where(np.isin(range(1,self.grid.dim+1),self.contains)==False)[0][0]+1
        empty_cell = np.where(np.isnan([c.value for c in self.cells]))[0][0]
        self.cells[empty_cell] = self.cells[empty_cell].set_value(missing_value)

      # if there is a number that can only go in one postion
      for v in range(self.grid.dim):
        can_array = np.array([c.can_be[v] for c in self.cells])
        if np.sum(can_array==True) == 1:
          c = np.where(can_array==True)[0][0]      
          self.cells[c].set_value(int(v+1))

    return self

  def refresh(self):
    if not self.grid.solved():

      # refresh the list of numbers already in this group
      for i,c in enumerate(self.cells):
        if not np.isnan(c.value):
          if not np.isin(c.value,self.contains):
            self.contains.append(c.value)
    return self
  
  def find_cells(self):
    pass
    # implemented differently in the base classes
    # this is essentially the only difference between rows, columns and squares

class column(group):
  def __init__(self,grid,number):
    group.__init__(self,grid,number)

  def find_cells(self):
    # only called on initialisation
    # assign cells to this column
    for i in range(self.grid.dim):
      self.cells.append(self.grid.cells[self.number+i*self.grid.dim])
    return self

class row(group):
  def __init__(self,grid,number):
    group.__init__(self,grid,number)

  def find_cells(self):
    # only called on initialisation
    # assign cells to this row
    for i in range(self.grid.dim):
      self.cells.append(self.grid.cells[self.number*self.grid.dim+i])
    return self

class square(group):
  def __init__(self,grid,number):
    group.__init__(self,grid,number)

  def find_cells(self):
    # only called on initialisation
    # assign cells to this column
    for c in self.grid.cells:
      # this is a bit of a copout - just find the cells that have this square as their square
      if c.square_no == self.number:
        self.cells.append(c)
    return self



Next we define some functions to define a grid and solve it.
The grids we will solve are stored in a csv file in the format 004300209005009001070060043006002087190007400050083000600000105003508690042910300 

Thus the assign function takes strings of this type as its argument

In [ ]:
# these functions use the classes in the previous cell

def assign(grid,values):
  for i,v in enumerate(values):
    if v != '0': 
      grid.cells[i] = grid.cells[i].set_value(int(v))
  
  return grid

def iterate(grid):

  for i,r in enumerate(grid.rows):
    grid.rows[i] = r.refresh()
    grid.rows[i] = r.check()

  for i,c in enumerate(grid.columns):
    grid.columns[i] = c.refresh()  
    grid.columns[i] = c.check()  

  for i,s in enumerate(grid.squares):
    grid.squares[i] = s.refresh()
    grid.squares[i] = s.check()

  for i,c in enumerate(grid.cells):
    grid.cells[i] = c.refresh()
    c.sanity()

  return grid


def solve_simple(grid,render=True):
  
  if render:
    output.clear()
    grid.render()
    time.sleep(1)

  grid = iterate(grid)
  last_str = grid.to_string()
  new_str = ''
  counter = 1
  while new_str != last_str:
    grid = iterate(grid)
    if render:
      time.sleep(1)
      output.clear()
      grid.render()  
    last_str = new_str
    new_str = grid.to_string()
    counter = counter + 1
    if grid.solved():
      break
    if grid.solvable == False:
      break
      
  return grid



Now lets see all this in action - first load the puzzles from the csv file to a dataframe

In [ ]:
all_grids = pd.read_csv('/sample_data/sudoku_examples.csv')

all_grids.shape

(155599, 2)

Now try a few - there are 155,599 examples

In [9]:
puzzle_no =155590
my_grid = grid(9)
my_grid = assign(my_grid,all_grids['quizzes'][puzzle_no])
my_grid = solve_simple(my_grid)

# alternatively we can see watch every update:
my_grid = grid(9)
my_grid = assign(my_grid,all_grids['quizzes'][puzzle_no])
my_grid.render_at_each_step = True
my_grid = solve_simple(my_grid,render=False)

5 6 1 |2 8 4 |7 3 9 
7 9 4 |1 3 6 |8 2 5 
2 3 8 |9 7 5 |4 6 1 
------+------+------
6 1 5 |4 9 8 |3 7 2 
8 4 7 |3 1 2 |9 5 6 
9 2 3 |5 6 7 |1 4 8 
------+------+------
1 5 2 |8 4 3 |6 9 7 
3 8 6 |7 5 9 |2 1 4 
4 7 9 |6 2 1 |5 8 3 


This simple algorithm won't solve every solvable grid. For incomplete grids we use a brute force approach, randomly picking a cell and trying a rabdin value. We apply this process recursively -  if we find an incomplete grid we randomly assign another cell. We continue until we find a full solution, or prove that the current grid is inconsistent, in which case we back up one level and try a different value. 

This sounds complicated, but the code is simple enough.

This approach is guaranteed to find a solution if it exists.


In [ ]:
def solve_full(grid,render=True):
  # this version handles grids that may not be solvable by the simple algorithm
  # if the simple algorithm cannot solve the problem we proceed to guessing

  grid = solve_simple(grid,render=False)
  
  if render:  
    output.clear()
    grid.render()
    time.sleep(1)

  if grid.solvable == False:
    return grid
  if grid.solved() == True:
    return grid

  grid_list = []
  value_try_list = []
  cell_no_list = []

  while grid.solved() == False and grid.solvable == True:
    if render:  
      output.clear()
      grid.render()
      time.sleep(1)

    empty_cells = np.where(np.isnan([c.value for c in grid.cells]))[0]
    pivot = np.random.choice(empty_cells)
    candidates = np.where(grid.cells[pivot].can_be)[0]+1
    value_try = np.random.choice(candidates)

    test_grid = deepcopy(grid)
    test_grid.cells[pivot].set_value(value_try)
    test_grid = solve_full(test_grid)

    if test_grid.solved():
      return test_grid

    if test_grid.solvable == False:
      grid.cells[pivot].can_be[value_try-1] = False
      #grid.cells[pivot] = grid.cells[pivot].refresh()
      grid = iterate(grid)


  return grid


Run the below example and you can see how the algorithm explores some avenues and then goes back to an ealier version of the grid when it encounters a contradiction.

Comment line 16 (my_grid.render_at_each_step = True) to run it more quickly. In general the speed of convergence is random because of the random choices in the algorithm.

In [10]:
filename = '/sample_data/sudoku_uploader.xlsx'
def excel_to_grid(filename):
  in_df = pd.read_excel(filename,usecols="B:J")
  s = ''
  for i in range(in_df.shape[0]):
    for j in range(in_df.shape[1]):
      if np.isnan(in_df.iloc[i,j]):
        s = s + '0'
      else:
        s = s + str(int(in_df.iloc[i,j]))
  my_grid = grid(9)
  my_grid = assign(my_grid,s)
  return my_grid

my_grid = excel_to_grid(filename)
#my_grid.render_at_each_step = True
my_grid.render()
my_grid = solve_full(my_grid, render = True)

if my_grid.solved():
  print('Solved!')

9 5 4 |1 3 7 |6 8 2 
2 7 3 |6 8 4 |1 9 8 
1 6 8 |9 2 5 |3 7 4 
------+------+------
4 9 5 |2 6 3 |7 5 1 
6 8 7 |4 5 1 |2 3 9 
3 2 1 |7 9 8 |5 4 6 
------+------+------
7 4 9 |3 1 2 |8 6 5 
5 1 6 |8 7 9 |4 2 3 
8 3 2 |5 4 6 |9 1 7 
Solved!


Finally, if you want you can try to solve all the puzzles...

In [8]:
output_df = pd.DataFrame(columns = ['Number','Solved','Solution','Correct','Time'])
# this will take a while ...
for i in tqdm(range(all_grids.shape[0])):
  my_grid = grid(9)
  my_grid = assign(my_grid,all_grids['quizzes'][i])
  start_time = time.time()
  my_grid = solve_simple(my_grid,False)
  output_df.loc[i] = [i,my_grid.solved(),my_grid.to_string(),all_grids['solutions'][i] == my_grid.to_string(),start_time - time.time()]

# all puzzles should be solved correctly
print(np.all(output_df.Correct==True))
print(np.all(output_df.Solved==True))


True
True
